In [1]:
import pandas as pd
import datetime as dt
import numpy as np
# importing the data
df1 = pd.read_csv( "olist_orders_dataset.csv")
df2=pd.read_csv("olist_order_items_dataset.csv")
df3=pd.read_csv("olist_products_dataset.csv")
df4=pd.merge(df1,df2,how='inner',on='order_id')
df=pd.merge(df3,df4,how='inner',on='product_id')

In [2]:
df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_id,...,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,e17e4f88e31525f7deef66779844ddce,...,2018-04-24 16:16:53,2018-04-24 19:04:19,2018-04-25 17:01:00,2018-04-27 16:42:17,2018-05-09 00:00:00,1,5670f4db5b62c43d542e1b2d56b0cf7c,2018-04-30 17:33:54,10.91,7.39
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,5236307716393b7114b53ee991f36956,...,2018-01-31 18:55:36,2018-01-31 19:11:15,2018-02-05 22:44:47,2018-02-16 13:27:08,2018-02-22 00:00:00,1,b561927807645834b59ef0d16ba55a24,2018-02-06 19:11:15,248.00,17.99
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,01f66e58769f84129811d43eefd187fb,...,2018-07-09 21:18:01,2018-07-09 21:30:20,2018-07-10 11:49:00,2018-07-11 23:24:16,2018-07-18 00:00:00,1,7b07b3c7487f0ea825fc6df75abd658b,2018-07-11 21:30:20,79.80,7.82
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,143d00a4f2dde4e0364ee1821577adb3,...,2018-08-03 08:55:50,2018-08-03 09:31:42,2018-08-03 11:48:00,2018-08-06 15:14:32,2018-08-08 00:00:00,1,c510bc1718f0f2961eaa42a23330681a,2018-08-07 09:10:13,112.30,9.54
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,86cafb8794cb99a9b1b77fc8e48fbbbb,...,2018-04-11 01:06:37,2018-04-11 01:30:23,2018-04-12 22:56:27,2018-04-13 22:41:44,2018-04-26 00:00:00,1,0be8ff43f22e456b4e0371b2245e4d01,2018-04-17 01:30:23,37.90,8.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,59e88b7d3675e89aceaf86f372d3bc9a,...,2018-01-03 15:05:53,2018-01-04 05:14:57,2018-01-04 15:57:18,2018-01-15 20:39:35,2018-02-02 00:00:00,2,4324dd16853115efb0fd9d0d131ba6f4,2018-01-10 03:32:09,29.90,16.11
112646,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,6e4465d771f02e4fe335225de3c6c043,...,2018-03-06 11:51:44,2018-03-08 02:49:35,2018-03-08 23:16:19,2018-03-23 20:08:40,2018-04-05 00:00:00,1,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28
112647,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,6e4465d771f02e4fe335225de3c6c043,...,2018-03-06 11:51:44,2018-03-08 02:49:35,2018-03-08 23:16:19,2018-03-23 20:08:40,2018-04-05 00:00:00,2,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28
112648,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,6e4465d771f02e4fe335225de3c6c043,...,2018-03-06 11:51:44,2018-03-08 02:49:35,2018-03-08 23:16:19,2018-03-23 20:08:40,2018-04-05 00:00:00,3,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28


In [3]:
df_recency = df.groupby(by='customer_id', 
                        as_index=False)['order_approved_at'].max()

In [4]:
df_recency.columns = ['customer_id', 'LastPurchaseDate']
df_recency["LastPurchaseDate"]=pd.to_datetime(df_recency["LastPurchaseDate"])
recent_date = df_recency['LastPurchaseDate'].max()

In [5]:
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)

In [7]:
frequency_df = df.drop_duplicates().groupby(
    by=['customer_id'], as_index=False)['order_approved_at'].count()
frequency_df.columns = ['customer_id', 'Frequency']

In [8]:
df['Total'] = df['price']* df['order_item_id']
monetary_df = df.groupby(by='customer_id', as_index=False)['Total'].sum()
monetary_df.columns = ['customer_id', 'Monetary']

In [9]:
rf_df = df_recency.merge(frequency_df, on='customer_id')
rfm_df = rf_df.merge(monetary_df, on='customer_id').drop(
	columns='LastPurchaseDate')

In [11]:
df=rfm_df.merge(df, on='customer_id')

In [12]:
df

,customer_id,Recency,Frequency,Monetary,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,...,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,Total
0,00012a2ce6f8dcda20d059ce98491703,293.0,1,89.80,64315bd8c0c47303179dd2e25b579d00,brinquedos,55.0,889.0,1.0,4267.0,...,2017-11-14 16:35:32,2017-11-17 15:32:08,2017-11-28 15:41:30,2017-12-04 00:00:00,1,7aa4334be125fcdd2ba64b3180029f14,2017-11-21 16:35:32,89.80,24.94,89.80
1,000161a058600d5901f007fab4c27140,414.0,1,54.90,84183944dc7cddca87a5d384452c1d3c,beleza_saude,57.0,2563.0,2.0,150.0,...,2017-07-16 09:55:12,2017-07-19 19:09:37,2017-07-25 18:57:33,2017-08-04 00:00:00,1,2a1348e9addc1af5aaa619b1a3679d6b,2017-07-20 09:55:12,54.90,12.51,54.90
2,0001fd6190edaaf884bcaf3d49edf079,552.0,1,179.99,9df2b21ec85378d71df4404712e17478,bebes,45.0,1042.0,2.0,750.0,...,2017-02-28 11:15:20,2017-03-01 15:24:20,2017-03-06 08:57:49,2017-03-22 00:00:00,1,46dc3b2cc0980fb8ec44634e21d2718e,2017-03-06 11:15:20,179.99,15.43,179.99
3,0002414f95344307404f0ace7a26f1d5,382.0,1,149.90,af3ec22cce878225aae6d9eb6c7a78eb,cool_stuff,38.0,840.0,1.0,1600.0,...,2017-08-17 03:10:27,2017-08-19 11:34:29,2017-09-13 20:06:02,2017-09-14 00:00:00,1,aafe36600ce604f205b86b5084d3d767,2017-08-23 03:10:27,149.90,29.45,149.90
4,000379cdec625522490c315e70c7a9fb,152.0,1,93.00,868b3136c5b206f91b8208fbfdf2cb7c,cama_mesa_banho,57.0,364.0,2.0,800.0,...,2018-04-04 03:10:19,2018-04-04 18:11:09,2018-04-13 20:21:08,2018-04-18 00:00:00,1,4a3ca9315b744ce9f8e9374361493884,2018-04-10 03:10:19,93.00,14.01,93.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffecc9f79fd8c764f843e9951b11341,158.0,1,54.90,c045b19fdb30a3035c56d04c6fe2e622,cama_mesa_banho,48.0,641.0,3.0,2050.0,...,2018-03-29 17:10:27,2018-03-31 14:29:38,2018-04-10 17:20:49,2018-04-27 00:00:00,1,259f7b5e6e482c230e5bfaa670b6bb8f,2018-04-04 17:10:27,54.90,26.46,54.90
112646,fffeda5b6d849fbd39689bb92087f431,104.0,1,47.90,d5c263f4d651ba657c900ce70a4be7de,telefonia,59.0,458.0,4.0,350.0,...,2018-05-22 13:54:37,2018-05-25 13:25:00,2018-06-08 18:03:31,2018-06-29 00:00:00,1,dd55f1bb788714a40e7954c3be6df745,2018-05-28 13:54:37,47.90,15.23,47.90
112647,ffff42319e9b2d713724ae527742af25,82.0,1,199.90,e61c78a7343d82c0539d27df0f7dfc31,cama_mesa_banho,60.0,268.0,1.0,2700.0,...,2018-06-13 17:20:23,2018-06-15 18:52:00,2018-06-18 18:33:05,2018-06-25 00:00:00,1,da8622b14eb17ae2831f4ac5b9dab84a,2018-06-15 17:20:23,199.90,14.23,199.90
112648,ffffa3172527f765de70084a7e53aae8,366.0,2,32.70,43ee88561093499d9e571d4db5f20b79,moveis_decoracao,39.0,161.0,3.0,200.0,...,2017-09-02 12:05:40,2017-09-08 20:04:11,2017-09-14 19:47:40,2017-09-26 00:00:00,1,23613d49c3ac2bd302259e55c06c050c,2017-09-08 12:05:40,10.90,11.85,10.90


In [13]:
df.columns

Index(['customer_id', 'Recency', 'Frequency', 'Monetary', 'product_id',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value', 'Total'],
      dtype='object')

In [14]:
df.drop(columns=['product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm','order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date','order_item_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value'],axis=1,inplace=True)

In [15]:
df

,customer_id,Recency,Frequency,Monetary,product_id,product_category_name,order_id,Total
0,00012a2ce6f8dcda20d059ce98491703,293.0,1,89.80,64315bd8c0c47303179dd2e25b579d00,brinquedos,5f79b5b0931d63f1a42989eb65b9da6e,89.80
1,000161a058600d5901f007fab4c27140,414.0,1,54.90,84183944dc7cddca87a5d384452c1d3c,beleza_saude,a44895d095d7e0702b6a162fa2dbeced,54.90
2,0001fd6190edaaf884bcaf3d49edf079,552.0,1,179.99,9df2b21ec85378d71df4404712e17478,bebes,316a104623542e4d75189bb372bc5f8d,179.99
3,0002414f95344307404f0ace7a26f1d5,382.0,1,149.90,af3ec22cce878225aae6d9eb6c7a78eb,cool_stuff,5825ce2e88d5346438686b0bba99e5ee,149.90
4,000379cdec625522490c315e70c7a9fb,152.0,1,93.00,868b3136c5b206f91b8208fbfdf2cb7c,cama_mesa_banho,0ab7fb08086d4af9141453c91878ed7a,93.00
...,...,...,...,...,...,...,...,...
112645,fffecc9f79fd8c764f843e9951b11341,158.0,1,54.90,c045b19fdb30a3035c56d04c6fe2e622,cama_mesa_banho,814d6a3a7c0b32b2ad929ac6328124e9,54.90
112646,fffeda5b6d849fbd39689bb92087f431,104.0,1,47.90,d5c263f4d651ba657c900ce70a4be7de,telefonia,8c855550908247a7eff50281b92167a8,47.90
112647,ffff42319e9b2d713724ae527742af25,82.0,1,199.90,e61c78a7343d82c0539d27df0f7dfc31,cama_mesa_banho,83b5fc912b2862c5046555ded1483ae9,199.90
112648,ffffa3172527f765de70084a7e53aae8,366.0,2,32.70,43ee88561093499d9e571d4db5f20b79,moveis_decoracao,d0e7be325a1c986babc4e1cdb91edc03,10.90


In [16]:
df.columns

Index(['customer_id', 'Recency', 'Frequency', 'Monetary', 'product_id',
       'product_category_name', 'order_id', 'Total'],
      dtype='object')

In [17]:
pivot_frequency = pd.pivot_table(data=df , index = "customer_id" , columns="product_category_name" , values ="Frequency" , aggfunc = len )

In [18]:
pivot_recency = pd.pivot_table(data=df , index = "customer_id" , columns="product_category_name" , values ="Recency" , aggfunc = min )

C:\Users\Hossam\AppData\Local\Temp\ipykernel_8224\230088399.py:1: FutureWarning: The provided callable <built-in function min> is currently using DataFrameGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  pivot_recency = pd.pivot_table(data=df , index = "customer_id" , columns="product_category_name" , values ="Recency" , aggfunc = min )


In [19]:
pivot_montery = pd.pivot_table(data=df , index = "customer_id" , columns="product_category_name" , values ="Monetary" , aggfunc = sum )

C:\Users\Hossam\AppData\Local\Temp\ipykernel_8224\1630321990.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_montery = pd.pivot_table(data=df , index = "customer_id" , columns="product_category_name" , values ="Monetary" , aggfunc = sum )


In [20]:
pivot_recency = pivot_recency.add_suffix("_recency")
pivot_frequency = pivot_frequency.add_suffix("_frequency")
pivot_montery = pivot_montery.add_suffix("_montery")

In [21]:
pivot_total = pd.concat([pivot_recency , pivot_frequency , pivot_montery] , axis = 1)

In [22]:
pivot_total.fillna( 0 , inplace = True)

In [23]:
pivot_total

product_category_name,agro_industria_e_comercio_recency,alimentos_recency,alimentos_bebidas_recency,artes_recency,artes_e_artesanato_recency,artigos_de_festas_recency,artigos_de_natal_recency,audio_recency,automotivo_recency,bebes_recency,...,pet_shop_montery,portateis_casa_forno_e_cafe_montery,portateis_cozinha_e_preparadores_de_alimentos_montery,relogios_presentes_montery,seguros_e_servicos_montery,sinalizacao_e_seguranca_montery,tablets_impressao_imagem_montery,telefonia_montery,telefonia_fixa_montery,utilidades_domesticas_montery
customer_id,,,,,,,,,,,,,,,,,,,,,
00012a2ce6f8dcda20d059ce98491703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000161a058600d5901f007fab4c27140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001fd6190edaaf884bcaf3d49edf079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,552.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002414f95344307404f0ace7a26f1d5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000379cdec625522490c315e70c7a9fb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74bebaf46603f9340e3b50c6b086f992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a3d3c38e58b9d2dfb9207cab690b6310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d5de688c321096d15508faae67a27051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN

In [25]:
steps = []
steps.append(("Scaler" , StandardScaler()))
steps.append(("Model" , DBSCAN(eps=3.25 , min_samples=250 , n_jobs = -1 )))
pipepline = Pipeline(steps = steps)

In [26]:
pipepline.fit(pivot_total)

c:\Users\Hossam\anaconda3\envs\churn\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\Hossam\anaconda3\envs\churn\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Hossam\anaconda3\envs\churn\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Hossam\anaconda3\envs\churn\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check 

Pipeline(steps=[('Scaler', StandardScaler()),
                ('Model', DBSCAN(eps=3.25, min_samples=250, n_jobs=-1))])

In [27]:
import numpy as np

In [28]:
np.unique(pipepline["Model"].labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18], dtype=int64)

In [29]:
pivot_total["Cluster"] = pipepline["Model"].labels_

In [30]:
pivot_total["Cluster"].value_counts()

Cluster
 1     37094
-1     14371
 14     6550
 12     5322
 6      3907
 13     3625
 0      3617
 3      3401
 8      2929
 5      2924
 2      2681
 11     2362
 9      2077
 4      1686
 10     1477
 16     1027
 7       791
 15      697
 17      388
 18      351
Name: count, dtype: int64

In [33]:
clus_map ={i:i+1 for i in pivot_total["Cluster"].unique()}
clus_map[-1]=20
clus_map

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 -1: 20,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 15: 16,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 18: 19,
 16: 17,
 17: 18}

In [34]:
pivot_total["Cluster"] = pivot_total["Cluster"].map(clus_map)

In [35]:
pivot_total["Cluster"].value_counts()

Cluster
2     37094
20    14371
15     6550
13     5322
7      3907
14     3625
1      3617
4      3401
9      2929
6      2924
3      2681
12     2362
10     2077
5      1686
11     1477
17     1027
8       791
16      697
18      388
19      351
Name: count, dtype: int64

In [36]:
import plotly.express as px

In [40]:
def Get_Categories_Values(cluster , dataframe):
    df_cluster = dataframe[dataframe["Cluster"] == cluster]
    df_categories = pd.DataFrame()
    for col in dataframe.columns:
        if col.split("_")[-1] == "montery":
            df_categories.at[0,col] = df_cluster[col].sum()
    df_categories = df_categories.transpose().reset_index().rename(columns={"index" : "Category" , 0 : "Value"})
    fig = px.histogram(data_frame= df_categories , x = "Category" , y ="Value" , title=f"Cluster {cluster} Categories")
    fig.update_xaxes(categoryorder = "total descending")
    fig.show()
    return df_categories

In [41]:
for i in range(1,21):
    Get_Categories_Values(cluster = i ,dataframe=pivot_total)

In [42]:
df_cluster = pivot_total[pivot_total["Cluster"] == 1]
df_categories = pd.DataFrame()
for col in pivot_total.columns:
    if col.split("_")[-1] == "montery":
        df_categories.at[0,col] = df_cluster[col].sum()

In [43]:
df_categories.transpose().reset_index().rename(columns={"index" : "Category" , 0 : "Value"})

,Category,Value
0,agro_industria_e_comercio_montery,0.0
1,alimentos_montery,0.0
2,alimentos_bebidas_montery,0.0
3,artes_montery,0.0
4,artes_e_artesanato_montery,0.0
...,...,...
68,sinalizacao_e_seguranca_montery,0.0
69,tablets_impressao_imagem_montery,0.0
70,telefonia_montery,0.0
71,telefonia_fixa_montery,0.0


In [44]:
pivot_total.reset_index(inplace = True)

In [46]:
pivot_total

product_category_name,customer_id,agro_industria_e_comercio_recency,alimentos_recency,alimentos_bebidas_recency,artes_recency,artes_e_artesanato_recency,artigos_de_festas_recency,artigos_de_natal_recency,audio_recency,automotivo_recency,...,portateis_casa_forno_e_cafe_montery,portateis_cozinha_e_preparadores_de_alimentos_montery,relogios_presentes_montery,seguros_e_servicos_montery,sinalizacao_e_seguranca_montery,tablets_impressao_imagem_montery,telefonia_montery,telefonia_fixa_montery,utilidades_domesticas_montery,Cluster
0,00012a2ce6f8dcda20d059ce98491703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,000161a058600d5901f007fab4c27140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0001fd6190edaaf884bcaf3d49edf079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0002414f95344307404f0ace7a26f1d5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,000379cdec625522490c315e70c7a9fb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97272,74bebaf46603f9340e3b50c6b086f992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
97273,a3d3c38e58b9d2dfb9207cab690b6310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
97274,d5de688c321096d15508faae67a27051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
97275,d85919cb3c0529589c6fa617f5f43281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [47]:
df = pd.merge(left= pivot_total[["customer_id" , "Cluster"]] , right=df , on="customer_id")

In [48]:
df

,customer_id,Cluster,Recency,Frequency,Monetary,product_id,product_category_name,order_id,Total
0,00012a2ce6f8dcda20d059ce98491703,1,293.0,1,89.80,64315bd8c0c47303179dd2e25b579d00,brinquedos,5f79b5b0931d63f1a42989eb65b9da6e,89.80
1,000161a058600d5901f007fab4c27140,2,414.0,1,54.90,84183944dc7cddca87a5d384452c1d3c,beleza_saude,a44895d095d7e0702b6a162fa2dbeced,54.90
2,0001fd6190edaaf884bcaf3d49edf079,3,552.0,1,179.99,9df2b21ec85378d71df4404712e17478,bebes,316a104623542e4d75189bb372bc5f8d,179.99
3,0002414f95344307404f0ace7a26f1d5,4,382.0,1,149.90,af3ec22cce878225aae6d9eb6c7a78eb,cool_stuff,5825ce2e88d5346438686b0bba99e5ee,149.90
4,000379cdec625522490c315e70c7a9fb,2,152.0,1,93.00,868b3136c5b206f91b8208fbfdf2cb7c,cama_mesa_banho,0ab7fb08086d4af9141453c91878ed7a,93.00
...,...,...,...,...,...,...,...,...,...
111114,a3d3c38e58b9d2dfb9207cab690b6310,6,NaN,0,135.00,4fd676d9c4723d475026e40aeae56957,ferramentas_jardim,2eecb0d85f281280f79fa00f9cec1a95,135.00
111115,d5de688c321096d15508faae67a27051,2,NaN,0,45.90,c3b271f47e73d0c9ccf1b43b7606c705,moveis_decoracao,7002a78c79c519ac54022d4f8a65e6e8,45.90
111116,d85919cb3c0529589c6fa617f5f43281,2,NaN,0,133.99,db8ed3d08891d16a2438a67ab3acb740,cama_mesa_banho,3c0b8706b065f9919d0505d3b3343881,133.99
111117,f67cd1a215aae2a1074638bbd35a223a,20,NaN,0,147.00,8c5876b1c7768217964f353bc7e64393,telefonia,88083e8f64d95b932164187484d90212,49.00


In [53]:
df[df["Cluster"] == 1].groupby("product_category_name")["Monetary"].sum().sort_values(ascending = False)

product_category_name
brinquedos         415850.36
cama_mesa_banho       199.88
esporte_lazer         115.79
Name: Monetary, dtype: float64

In [63]:
def Get_Recommendation(User , dataframe , num_of_recommendations):
    Cluster = dataframe[dataframe["customer_id"] == str(User)].reset_index()["Cluster"][0]
    print(Cluster)
    df_cluster = dataframe[dataframe["Cluster"] == Cluster]
    df_cluster = df_cluster.groupby("product_category_name")["Monetary"].sum().sort_values(ascending = False).head(int(num_of_recommendations))
    for index , name in enumerate(df_cluster.index) : 
        print(f"Recommendation number {index + 1} is {name}")

In [66]:
Get_Recommendation("00012a2ce6f8dcda20d059ce98491703",df,3)

1
Recommendation number 1 is brinquedos
Recommendation number 2 is cama_mesa_banho
Recommendation number 3 is esporte_lazer


In [62]:
df.to_csv("Clustered_data.csv" , index = False)